# Training Prediction Models Directly Within PostgreSQL Using EvaDB
In this tutorial, we'll harness EvaDB's model training capabilities to predict home rental prices, showcasing how EvaDB seamlessly integrates AI into your PostgreSQL database.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

## Setup

### Install and Launch the PostgreSQL Server

To kick things off, we'll start by setting up the PostgreSQL database backend. If you already have a PostgreSQL server up and running, you can skip this step and proceed directly to [installing EvaDB](#install-evadb).

In [1]:
!apt -qq install postgresql
!service postgresql start

The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
0 upgraded, 13 newly installed, 0 to remove and 18 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package logrotate.
(Reading database ... 120901 files and directories currently installed.)
Preparing to unpack .../00-logrotate_3.19.0-1ubuntu1.1_amd64.deb ...
Unpacking logrotate (3.19.0-1ubuntu1

### Create User and Database

In [2]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


### Prettify  Output

In [3]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
def pretty_print(df):
    return display(HTML( df.to_html().replace("\\n","<br>")))

### Installing EvaDB
<a id='install_evadb'></a>
We install EvaDB along with the necessary PostgreSQL and Ludwig dependencies.

In [4]:
%pip install --quiet "evadb[postgres,ludwig]"

import evadb
cursor = evadb.connect().cursor()

## Load data into PostgresSQL


### Setting up a Data Source in EvaDB
To establish a direct connection between EvaDB and underlying database systems such as PostgreSQL, we will create a data source. This process entails supplying EvaDB with the connection credentials for the active PostgreSQL server.

In [6]:
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

,0
0,The database postgres_data has been successful...


### Loading Home Property Sales Data from CSV into PostgreSQL

In this step, we will import the [House Property Sales](https://www.kaggle.com/datasets/htagholdings/property-sales?resource=download) dataset into our PostgreSQL database. If you already have the data stored in PostgreSQL and are ready to proceed with the prediction model training, feel free to skip this section and head directly to the [model training process](#train-the-prediction-model).


In [ ]:
!mkdir -p content
!wget -nc -O /content/home_rentals.csv https://www.dropbox.com/scl/fi/gy2682i66a8l2tqsowm5x/home_rentals.csv?rlkey=e080k02rv5205h4ullfjdr8lw&raw=1

In [8]:
cursor.query("""
  USE postgres_data {
    CREATE TABLE IF NOT EXISTS home_rentals (
      number_of_rooms INT,
      number_of_bathrooms INT,
      sqft INT,
      location VARCHAR(128),
      days_on_market INT,
      initial_price INT,
      neighborhood VARCHAR(128),
      rental_price FLOAT
    )
  }
""").df()

,status
0,success


In [27]:
cursor.query("""
  USE postgres_data {
    COPY home_rentals(number_of_rooms, number_of_bathrooms, sqft, location, days_on_market, initial_price, neighborhood, rental_price)
    FROM '/content/home_rentals.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

,status
0,success


### Preview the Data


Within the home_rentals table, there are 8 columns at our disposal. Our objective is to utilize the remaining 7 columns to make predictions for the rental_price.

In [26]:
cursor.query("SELECT * FROM postgres_data.home_rentals LIMIT 3;").df()

/nethome/gkakkar7/VAST/evadb/evadb_310/lib/python3.10/site-packages/evadb/third_party/databases/postgres/postgres_handler.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/nethome/gkakkar7/VAST/evadb/evadb_310/lib/python3.10/site-packages/evadb/third_party/databases/postgres/postgres_handler.py:110: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)


,home_rentals.number_of_rooms,home_rentals.number_of_bathrooms,home_rentals.sqft,home_rentals.location,home_rentals.days_on_market,home_rentals.initial_price,home_rentals.neighborhood,home_rentals.rental_price
0,1,1,674,good,1,2167,downtown,2167.0
1,1,1,554,poor,19,1883,westbrae,1883.0
2,0,1,529,great,3,2431,south_side,2431.0


## Training Model

Next, we employ EvaDB to facilitate the training of an ML model, which will enable us to predict `home rental prices`.

### Train the prediction Model
For this purpose, we harness the capabilities of the [ludwig](https://ludwig.ai/latest/) engine to train our prediction model. We employ the `automl` feature to automatically determine the optimal hyperparameters. Keep in mind that `TIME_LIMIT` specifies the time budget allocated for the training process.

In [18]:
cursor.query("""
  CREATE OR REPLACE FUNCTION PredictHouseRent FROM
  ( SELECT * FROM postgres_data.home_rentals )
  TYPE Ludwig
  PREDICT 'rental_price'
  TIME_LIMIT 3600;
""").df()

Analyzing fields: 100%|██████████| 8/8 [00:00<00:00, 328.58it/s]


(pid=9427) 2023-09-19 08:11:04.568610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(pid=9427) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


(pid=9427) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(pid=9427)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(pid=9427) Created a temporary directory at /tmp/tmpog4p303_
(pid=9427) Writing /tmp/tmpog4p303_/_remote_module_non_scriptable.py


...

(pid=14699) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
(pid=14699) Created a temporary directory at /tmp/tmp6bu1pg6f
(pid=14699) Writing /tmp/tmp6bu1pg6f/_remote_module_non_scriptable.py


(run_experiment_trial pid=14699) /usr/local/lib/python3.10/dist-packages/ludwig/schema/model_types/utils.py:238: UserWarning: Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.
(run_experiment_trial pid=14699)   warnings.warn("Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.")
(pid=15203) 2023-09-19 08:31:33.486977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(pid=15203) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(pid=15203)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(pid=15203) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
(pid=15203) Created a temporary directory at /tmp/tmpaend1ws3
(pid=15203) Writing /tmp/tmpaend1ws3/_remote_module_non_scriptable.py


(run_experiment_trial pid=15203) /usr/local/lib/python3.10/dist-packages/ludwig/schema/model_types/utils.py:238: UserWarning: Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.
(run_experiment_trial pid=15203)   warnings.warn("Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.")
2023-09-19 08:38:34,918	INFO tune.py:945 -- Total run time: 1663.01 seconds (1662.92 seconds for the tuning loop).


,0
0,Function PredictHouseRent overwritten.


### Utilizing the Prediction Model
Following the model training, we proceed to employ the `PredictHouseRent`` model to make predictions for home rental prices.

In [19]:
cursor.query("SELECT PredictHouseRent(*) FROM postgres_data.home_rentals LIMIT 10;").df()

2023-09-19 08:39:27,803	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(BatchInferModel)]
2023-09-19 08:39:27,805	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-09-19 08:39:27,861	INFO actor_pool_map_operator.py:99 -- MapBatches(BatchInferModel): Waiting for 1 pool actors to start...


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(MapWorker(MapBatches(BatchInferModel)) pid=17384) 2023-09-19 08:39:42.144391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(MapWorker(MapBatches(BatchInferModel)) pid=17384) Writing /tmp/tmpn1s5jolp/_remote_module_non_scriptable.py
(MapWorker(MapBatches(BatchInferModel)) pid=17384) Writing /tmp/tmpn1s5jolp/_remote_module_non_scriptable.py
(MapWorker(MapBatches(BatchInferModel)) pid=17384) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


(MapWorker(MapBatches(BatchInferModel)) pid=17384) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(MapWorker(MapBatches(BatchInferModel)) pid=17384)   warn("The installed version of bitsandbytes was compiled without GPU support. "
2023-09-19 08:39:58,956	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(postprocess_batch)]
2023-09-19 08:39:58,965	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

After compute: <class 'pandas.core.frame.DataFrame'>


,predicthouserent.rental_price_predictions
0,2087.763672
1,1793.570190
2,2346.319824
3,5551.257812
4,2333.635498
5,4354.810547
6,2121.776367
7,2107.004639
8,3906.090332
9,2026.162720


We have the option to utilize a `LATERAL JOIN` to compare the actual rental prices in the `home_rentals` dataset with the predicted rental prices generated by the trained model, `PredictHouseRent`.

In [20]:
cursor.query("""
  SELECT rental_price, predicted_rental_price FROM postgres_data.home_rentals
  JOIN LATERAL PredictHouseRent(*) AS Predicted(predicted_rental_price) LIMIT 10;
""").df()

2023-09-19 08:40:05,466	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(BatchInferModel)]
2023-09-19 08:40:05,469	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-09-19 08:40:05,515	INFO actor_pool_map_operator.py:99 -- MapBatches(BatchInferModel): Waiting for 1 pool actors to start...


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(MapWorker(MapBatches(BatchInferModel)) pid=17588) 2023-09-19 08:40:14.772637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(MapWorker(MapBatches(BatchInferModel)) pid=17588) Created a temporary directory at /tmp/tmpu8pnqblx
(MapWorker(MapBatches(BatchInferModel)) pid=17588) Writing /tmp/tmpu8pnqblx/_remote_module_non_scriptable.py


(MapWorker(MapBatches(BatchInferModel)) pid=17588) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(MapWorker(MapBatches(BatchInferModel)) pid=17588)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(MapWorker(MapBatches(BatchInferModel)) pid=17588) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2023-09-19 08:40:32,611	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(postprocess_batch)]
2023-09-19 08:40:32,613	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

After compute: <class 'pandas.core.frame.DataFrame'>


,home_rentals.rental_price,Predicted.predicted_rental_price
0,2167.000,2087.763672
1,1883.000,1793.570190
2,2431.000,2346.319824
3,5510.000,5551.257812
4,2272.000,2333.635498
5,4123.812,4354.810547
6,2224.000,2121.776367
7,2104.000,2107.004639
8,3861.000,3906.090332
9,2041.000,2026.162720
